<a href="https://colab.research.google.com/github/namanphy/EVA5/blob/main/S4/eva_session4_architectural_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

!pip install torchsummary
from torchsummary import summary

In [33]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 16, 3, padding=1), # RF - 3x3
            nn.ReLU(), # RF - 3x3
            # nn.BatchNorm2d(16), # RF - 3x3
            nn.Conv2d(16, 16, 3), # RF - 5x5
            nn.ReLU(), # RF - 5x5
            nn.BatchNorm2d(16), # RF - 5x5
            nn.MaxPool2d(2, 2) # RF - 10x10
        )

        self.conv2 = nn.Sequential(
            nn.Conv2d(16, 32, 3, padding=1), # RF - 12x12
            nn.ReLU(), # RF - 12x12
            # nn.BatchNorm2d(32), # RF - 12x12
            nn.Conv2d(32, 32, 3), # RF - 14x14
            nn.ReLU(), # RF - 14x14
            nn.BatchNorm2d(32), # RF - 14x14
            nn.MaxPool2d(2, 2) # RF - 28x28
        )
        self.conv3 = nn.Sequential(
            nn.Conv2d(32, 10, 3) # RF - 30x30
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)

        x = x.mean(dim=[-2,-1])
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [34]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
              ReLU-2           [-1, 16, 28, 28]               0
            Conv2d-3           [-1, 16, 26, 26]           2,320
              ReLU-4           [-1, 16, 26, 26]               0
       BatchNorm2d-5           [-1, 16, 26, 26]              32
         MaxPool2d-6           [-1, 16, 13, 13]               0
            Conv2d-7           [-1, 32, 13, 13]           4,640
              ReLU-8           [-1, 32, 13, 13]               0
            Conv2d-9           [-1, 32, 11, 11]           9,248
             ReLU-10           [-1, 32, 11, 11]               0
      BatchNorm2d-11           [-1, 32, 11, 11]              64
        MaxPool2d-12             [-1, 32, 5, 5]               0
           Conv2d-13             [-1, 10, 3, 3]           2,890
Total params: 19,354
Trainable params: 

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [8]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw




Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [28]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader, position=0, leave=True)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [29]:
data, target = next(iter(train_loader))
print(data.size(), target.size())

torch.Size([128, 1, 28, 28]) torch.Size([128])


In [35]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 18):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0559, Accuracy: 9840/10000 (98%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0374, Accuracy: 9896/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0215, Accuracy: 9930/10000 (99%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0231, Accuracy: 9928/10000 (99%)



loss=0.000600671221036464 batch_id=468: 100%|██████████| 469/469 [00:21<00:00, 21.46it/s]



Test set: Average loss: 0.0226, Accuracy: 9931/10000 (99%)



Max accuracy - 99.31%